In [1]:
# ================================================================
# V5 (FINAL): N1-PUSH upgrade on top of your V4 (minimal changes)
# Adds ONLY:
#   (1) Soft labels near transitions (boundary soft targets)
#   (2) Confusion-cost matrix regularizer (cost-sensitive penalty)
#   (3) Duration-aware auxiliary head (predict remaining run-length bucket)
# Keeps EVERYTHING else (encoder/transformer/EMA/LA-CE/aux N1/trans loss/smoothing)
# SHHS2 stays external eval. MESA is eval-only if present in manifest.
# ================================================================

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from collections import Counter
import math
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score, average_precision_score

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Visible CUDA devices:", torch.cuda.device_count())
print("Using device:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))


CUDA available: True
Visible CUDA devices: 1
Using device: cuda
GPU: NVIDIA RTX A6000


In [2]:
# ----------------------------
# Paths / Manifest
# ----------------------------
ROOT = Path("/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/")
MANIFEST_PATH = ROOT / "manifest_sleepstaging_planA.csv"

manifest = pd.read_csv(MANIFEST_PATH)
print("Rows:", len(manifest))
print(manifest.groupby(["cohort","split"]).size())

df_train = manifest[(manifest.cohort=="SHHS1") & (manifest.split=="train")].copy()
df_val   = manifest[(manifest.cohort=="SHHS1") & (manifest.split=="val")].copy()
df_test  = manifest[(manifest.cohort=="SHHS1") & (manifest.split=="test")].copy()
df_ext   = manifest[(manifest.cohort=="SHHS2") & (manifest.split=="external_test")].copy()

# Optional: MESA eval-only if exists in manifest
df_mesa = manifest[(manifest.cohort=="MESA")].copy() if ("MESA" in manifest["cohort"].unique()) else None
if df_mesa is not None and len(df_mesa) > 0:
    # prefer explicit split name if you used it; fallback to all MESA rows
    if "external_test" in df_mesa["split"].unique():
        df_mesa = df_mesa[df_mesa.split=="external_test"].copy()
    print("MESA detected | rows:", len(df_mesa))
else:
    df_mesa = None
    print("MESA not detected in manifest (ok).")

print("DF sizes:", len(df_train), len(df_val), len(df_test), len(df_ext), ("| MESA="+str(len(df_mesa)) if df_mesa is not None else ""))


Rows: 9868
cohort  split        
MESA    external_test    1856
SHHS1   test              548
        train            4380
        val               548
SHHS2   external_test    2536
dtype: int64
MESA detected | rows: 1856
DF sizes: 4380 548 548 2536 | MESA=1856


In [3]:
# ----------------------------
# Augment + Normalize (same as V4)
# ----------------------------
class EEGAugment:
    """
    Safe EEG augmentations on epochs.
    x: np.ndarray float32, shape (E, T)
    """
    def __init__(self,
                 p_amp=0.5,
                 p_noise=0.5,
                 p_shift=0.5,
                 p_bandstop=0.3,
                 p_freqdrop=0.3,
                 amp_range=(0.8, 1.2),
                 noise_std=0.01,       # relative to per-epoch std
                 shift_max=125,        # 1 sec at 125Hz
                 bandstop_ranges=((49,51), (59,61)),
                 freqdrop_max_bins=12):
        self.p_amp = p_amp
        self.p_noise = p_noise
        self.p_shift = p_shift
        self.p_bandstop = p_bandstop
        self.p_freqdrop = p_freqdrop
        self.amp_range = amp_range
        self.noise_std = noise_std
        self.shift_max = shift_max
        self.bandstop_ranges = bandstop_ranges
        self.freqdrop_max_bins = freqdrop_max_bins

    def _amp_scale(self, x):
        s = np.random.uniform(self.amp_range[0], self.amp_range[1])
        return x * s

    def _gaussian_noise(self, x):
        std = np.std(x, axis=1, keepdims=True) + 1e-6
        noise = np.random.randn(*x.shape).astype(np.float32) * (self.noise_std * std)
        return x + noise

    def _time_shift(self, x):
        shift = np.random.randint(-self.shift_max, self.shift_max+1)
        return np.roll(x, shift=shift, axis=1)

    def _bandstop_fft(self, x, fs=125.0):
        E, T = x.shape
        X = np.fft.rfft(x, axis=1)
        freqs = np.fft.rfftfreq(T, d=1.0/fs)
        for (f1, f2) in self.bandstop_ranges:
            mask = (freqs >= f1) & (freqs <= f2)
            X[:, mask] = 0.0
        y = np.fft.irfft(X, n=T, axis=1).astype(np.float32)
        return y

    def _freq_dropout(self, x):
        E, T = x.shape
        X = np.fft.rfft(x, axis=1)
        Fbins = X.shape[1]
        drop = np.random.randint(1, self.freqdrop_max_bins+1)
        start = np.random.randint(0, max(1, Fbins - drop))
        X[:, start:start+drop] = 0.0
        y = np.fft.irfft(X, n=T, axis=1).astype(np.float32)
        return y

    def __call__(self, x, fs=125.0):
        if np.random.rand() < self.p_amp:
            x = self._amp_scale(x)
        if np.random.rand() < self.p_noise:
            x = self._gaussian_noise(x)
        if np.random.rand() < self.p_shift:
            x = self._time_shift(x)
        if np.random.rand() < self.p_bandstop:
            x = self._bandstop_fft(x, fs=fs)
        if np.random.rand() < self.p_freqdrop:
            x = self._freq_dropout(x)
        return x

def normalize_epochs_zscore(x, eps=1e-6, clip=10.0):
    """
    per-epoch z-score: (x - mean) / std
    x: (E,T) float32 -> (E,T) float32
    """
    mu = np.mean(x, axis=1, keepdims=True)
    sd = np.std(x, axis=1, keepdims=True) + eps
    x = (x - mu) / sd
    if clip is not None:
        x = np.clip(x, -clip, clip)
    return x.astype(np.float32)

augment = EEGAugment()
print("Augmenter + Normalizer ready (per-epoch z-score).")


Augmenter + Normalizer ready (per-epoch z-score).


In [4]:
# ----------------------------
# Labels / constants
# ----------------------------
LABELS = {0:"W", 1:"N1", 2:"N2", 3:"N3", 4:"REM"}
NUM_CLASSES = 5
FS = 125
T = 3750   # 30s * 125Hz


In [5]:
# ================================================================
# V5 DATASET: same as V4, but returns extra supervision:
#   - soft_targets (L,C): boundary soft labels
#   - dur_bucket (L,): remaining run-length bucket labels
# ================================================================

def _compute_runlength_remaining(y):
    """
    y: (E,) int64
    returns remaining run length at each t: rem[t] = how many epochs left (including t) in the current segment
    """
    E = len(y)
    rem = np.zeros((E,), dtype=np.int64)
    t = 0
    while t < E:
        j = t
        while j < E and y[j] == y[t]:
            j += 1
        seg_len = j - t
        for k in range(t, j):
            rem[k] = (j - k)  # including current epoch
        t = j
    return rem

def _bucketize_remaining(rem, edges=(2, 5, 10, 20, 40, 80, 160)):
    """
    rem: (E,) remaining run length
    edges define bucket upper bounds.
    returns bucket id in [0..len(edges)] (B buckets)
    """
    edges = np.array(edges, dtype=np.int64)
    b = np.zeros_like(rem, dtype=np.int64)
    for i, r in enumerate(rem):
        b[i] = int(np.searchsorted(edges, r, side="right"))
    return b

def _make_soft_targets_boundary(y, num_classes=5, radius=2, alpha_max=0.35):
    """
    Soft labels near transitions:
      - if an epoch is within `radius` of a transition, mix current label with neighbor (prev/next).
      - alpha decays with distance: alpha = alpha_max * (1 - dist/radius)
    y: (E,)
    returns soft: (E,C) float32, sums to 1
    """
    E = len(y)
    soft = np.zeros((E, num_classes), dtype=np.float32)
    soft[np.arange(E), y] = 1.0

    if E < 2 or radius <= 0:
        return soft

    trans = np.where(y[1:] != y[:-1])[0] + 1  # boundary index (start of new stage)
    if trans.size == 0:
        return soft

    # For each boundary t (new stage begins at t), affect indices near t and t-1
    for t in trans:
        for dt in range(-radius, radius+1):
            i = t + dt
            if i < 0 or i >= E:
                continue
            dist = abs(dt)
            if dist > radius:
                continue
            alpha = alpha_max * (1.0 - (dist / max(radius, 1)))
            if alpha <= 0:
                continue

            # choose neighbor label towards the boundary direction
            if i < t:
                # before boundary => mix with next label y[t]
                nb = y[t]
            else:
                # at/after boundary => mix with prev label y[t-1]
                nb = y[t-1]

            cur = y[i]
            if nb == cur:
                continue

            # mix: (1-alpha)*onehot(cur) + alpha*onehot(nb)
            soft[i, :] = 0.0
            soft[i, cur] = 1.0 - alpha
            soft[i, nb]  = alpha

    # normalize (safety)
    soft = soft / (soft.sum(axis=1, keepdims=True) + 1e-8)
    return soft.astype(np.float32)

class SleepSequenceDataset(Dataset):
    """
    One item = one subject recording.

    Returns:
      x: (L, 1, T) float32
      y: (L,) int64
      mask: (L,) bool (True valid)
      soft: (L,C) float32 (soft targets near transitions)
      dur_bucket: (L,) int64 (duration-aware label)
    """
    def __init__(self, df, mode="train",
                 max_hours=None,
                 min_hours=2.0,
                 augmentor=None,
                 exclude_unknown=True,
                 do_normalize=True,
                 # ---- V4 additions ----
                 boundary_oversample_p=0.70,
                 boundary_radius=2,
                 # ---- V5 additions ----
                 soft_radius=2,
                 soft_alpha_max=0.35,
                 dur_edges=(2,5,10,20,40,80,160)):
        self.paths = df["npz_path"].tolist()
        self.mode = mode
        self.max_hours = max_hours
        self.min_hours = min_hours
        self.augmentor = augmentor
        self.exclude_unknown = exclude_unknown
        self.do_normalize = do_normalize

        self.boundary_oversample_p = boundary_oversample_p
        self.boundary_radius = int(boundary_radius)

        self.soft_radius = int(soft_radius)
        self.soft_alpha_max = float(soft_alpha_max)
        self.dur_edges = tuple(dur_edges)

        print(f"SleepSequenceDataset[{mode}] files={len(self.paths)} max_hours={self.max_hours} normalize={self.do_normalize}")

    def __len__(self):
        return len(self.paths)

    def _pick_block_start_boundary_aware(self, y, L):
        E = len(y)
        if E <= L:
            return 0

        if np.random.rand() > self.boundary_oversample_p:
            return np.random.randint(0, E - L + 1)

        trans = np.where(y[1:] != y[:-1])[0] + 1
        if trans.size == 0:
            return np.random.randint(0, E - L + 1)

        t = int(np.random.choice(trans))
        start = t - (L // 2)
        start = max(0, min(start, E - L))
        return int(start)

    def __getitem__(self, idx):
        p = self.paths[idx]
        d = np.load(p, allow_pickle=True)

        x = d["x"].astype(np.float32)    # (E,T)
        y = d["y"].astype(np.int64)      # (E,)

        if self.exclude_unknown:
            keep = (y >= 0)
            x = x[keep]
            y = y[keep]

        if self.do_normalize:
            x = normalize_epochs_zscore(x, eps=1e-6, clip=10.0)

        E = len(y)

        # V5: compute soft targets + duration labels BEFORE block crop,
        # then crop consistently (so supervision matches x,y)
        soft = _make_soft_targets_boundary(y, num_classes=NUM_CLASSES,
                                           radius=self.soft_radius,
                                           alpha_max=self.soft_alpha_max)
        rem = _compute_runlength_remaining(y)
        dur_bucket = _bucketize_remaining(rem, edges=self.dur_edges)

        # sample long block for training
        if self.max_hours is not None:
            max_L = int((self.max_hours * 3600) / 30)
            min_L = int((self.min_hours * 3600) / 30)

            L = min(max_L, E)
            if E > L:
                start = self._pick_block_start_boundary_aware(y, L)
                x = x[start:start+L]
                y = y[start:start+L]
                soft = soft[start:start+L]
                dur_bucket = dur_bucket[start:start+L]
                E = L

        if self.mode == "train" and self.augmentor is not None:
            x = self.augmentor(x, fs=FS)

        x_t = torch.from_numpy(x).unsqueeze(1)              # (E,1,T)
        y_t = torch.from_numpy(y).long()                    # (E,)
        soft_t = torch.from_numpy(soft).float()             # (E,C)
        dur_t = torch.from_numpy(dur_bucket).long()         # (E,)
        mask = torch.ones((E,), dtype=torch.bool)

        return x_t, y_t, mask, soft_t, dur_t

def collate_pad(batch):
    lengths = [b[0].shape[0] for b in batch]
    Lmax = max(lengths)

    xs, ys, ms, ss, ds = [], [], [], [], []
    for x, y, m, s, d in batch:
        L = x.shape[0]
        padL = Lmax - L
        if padL > 0:
            x = torch.cat([x, torch.zeros((padL, 1, T), dtype=x.dtype)], dim=0)
            y = torch.cat([y, torch.zeros((padL,), dtype=y.dtype)], dim=0)
            m = torch.cat([m, torch.zeros((padL,), dtype=torch.bool)], dim=0)
            s = torch.cat([s, torch.zeros((padL, NUM_CLASSES), dtype=s.dtype)], dim=0)
            d = torch.cat([d, torch.zeros((padL,), dtype=d.dtype)], dim=0)
        xs.append(x); ys.append(y); ms.append(m); ss.append(s); ds.append(d)

    x = torch.stack(xs, dim=0)  # (B,L,1,T)
    y = torch.stack(ys, dim=0)  # (B,L)
    m = torch.stack(ms, dim=0)  # (B,L)
    s = torch.stack(ss, dim=0)  # (B,L,C)
    d = torch.stack(ds, dim=0)  # (B,L)
    return x, y, m, s, d


In [6]:
# ----------------------------
# DataLoaders (same batching)
# ----------------------------
train_seq_ds = SleepSequenceDataset(
    df_train, mode="train",
    max_hours=4.0, min_hours=2.0,
    augmentor=augment,
    exclude_unknown=True,
    do_normalize=True,
    boundary_oversample_p=0.70,
    boundary_radius=2,
    # V5
    soft_radius=2,
    soft_alpha_max=0.35,
    dur_edges=(2,5,10,20,40,80,160),
)

val_seq_ds  = SleepSequenceDataset(df_val,  mode="eval", max_hours=None, augmentor=None, do_normalize=True)
test_seq_ds = SleepSequenceDataset(df_test, mode="eval", max_hours=None, augmentor=None, do_normalize=True)
ext_seq_ds  = SleepSequenceDataset(df_ext,  mode="eval", max_hours=None, augmentor=None, do_normalize=True)
mesa_seq_ds = SleepSequenceDataset(df_mesa, mode="eval", max_hours=None, augmentor=None, do_normalize=True) if df_mesa is not None else None

BATCH_SUBJ = 2
NUM_WORKERS = 2
PIN = True

train_seq_loader = DataLoader(
    train_seq_ds, batch_size=BATCH_SUBJ, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=PIN,
    collate_fn=collate_pad, persistent_workers=False
)

val_seq_loader  = DataLoader(val_seq_ds,  batch_size=1, shuffle=False, num_workers=1, pin_memory=PIN, collate_fn=collate_pad)
test_seq_loader = DataLoader(test_seq_ds, batch_size=1, shuffle=False, num_workers=1, pin_memory=PIN, collate_fn=collate_pad)
ext_seq_loader  = DataLoader(ext_seq_ds,  batch_size=1, shuffle=False, num_workers=1, pin_memory=PIN, collate_fn=collate_pad)
mesa_seq_loader = DataLoader(mesa_seq_ds, batch_size=1, shuffle=False, num_workers=1, pin_memory=PIN, collate_fn=collate_pad) if mesa_seq_ds is not None else None

xb, yb, mb, sb, db = next(iter(train_seq_loader))
print("Batch shapes:", xb.shape, yb.shape, mb.shape, sb.shape, db.shape)
print("Valid tokens:", int(mb.sum().item()), "| y min/max(valid):",
      int(yb[mb].min().item()), int(yb[mb].max().item()))


SleepSequenceDataset[train] files=4380 max_hours=4.0 normalize=True
SleepSequenceDataset[eval] files=548 max_hours=None normalize=True
SleepSequenceDataset[eval] files=548 max_hours=None normalize=True
SleepSequenceDataset[eval] files=2536 max_hours=None normalize=True
SleepSequenceDataset[eval] files=1856 max_hours=None normalize=True
Batch shapes: torch.Size([2, 480, 1, 3750]) torch.Size([2, 480]) torch.Size([2, 480]) torch.Size([2, 480, 5]) torch.Size([2, 480])
Valid tokens: 960 | y min/max(valid): 0 4


In [7]:
# ----------------------------
# Class weights (same as V4)
# ----------------------------
def class_counts_train(df):
    c = Counter()
    for p in tqdm(df["npz_path"].tolist(), desc="Counting train labels", leave=False):
        d = np.load(p, allow_pickle=True)
        y = d["y"].astype(np.int64)
        y = y[y >= 0]
        c.update(y.tolist())
    counts = np.array([c.get(i, 0) for i in range(NUM_CLASSES)], dtype=np.float64)
    return counts

counts = class_counts_train(df_train)
total = counts.sum()
weights = total / (NUM_CLASSES * np.maximum(counts, 1.0))
weights = weights / weights.mean()

print("Counts:", {LABELS[i]: int(counts[i]) for i in range(NUM_CLASSES)})
print("Weights:", {LABELS[i]: float(weights[i]) for i in range(NUM_CLASSES)})

class_weights = torch.tensor(weights, dtype=torch.float32).to(device)


Counts: {'W': 951298, 'N1': 159157, 'N2': 1801005, 'N3': 557286, 'REM': 620992}
Weights: {'W': 0.4653661355903559, 'N1': 2.781541962055294, 'N2': 0.24580824265053922, 'N3': 0.7943890104090797, 'REM': 0.7128946492947323}


In [8]:
# %% [code]
# ================================================================
# Model (V5.1): SAME V5 encoder + heads
# + Local-Global Attention (windowed most blocks, global every k blocks)
# Goal: better MESA generalization with small compute increase (vs full V6)
# ================================================================

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# -------------------------
# DropPath
# -------------------------
class DropPath(nn.Module):
    def __init__(self, drop_prob=0.1):
        super().__init__()
        self.drop_prob = float(drop_prob)

    def forward(self, x):
        if (not self.training) or self.drop_prob == 0.0:
            return x
        keep = 1.0 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        rand = keep + torch.rand(shape, device=x.device)
        mask = torch.floor(rand)
        return x / keep * mask

# -------------------------
# Residual Conv Block
# -------------------------
class ResConv1D(nn.Module):
    def __init__(self, c_in, c_out, k, s=1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(c_in, c_out, k, stride=s, padding=k//2),
            nn.BatchNorm1d(c_out),
            nn.GELU(),
            nn.Conv1d(c_out, c_out, k, padding=k//2),
            nn.BatchNorm1d(c_out),
        )
        self.skip = nn.Conv1d(c_in, c_out, 1, stride=s) if (c_in != c_out or s != 1) else nn.Identity()
        self.act = nn.GELU()

    def forward(self, x):
        return self.act(self.conv(x) + self.skip(x))

# -------------------------
# Epoch Encoder (FFT SAFE)
# -------------------------
class EpochEncoder(nn.Module):
    """
    Input : (B,L,1,T)
    Output: (B,L,384)
    """
    def __init__(self, d_model=384):
        super().__init__()
        self.branch_short = ResConv1D(1, 128, k=7,  s=4)
        self.branch_mid   = ResConv1D(1, 128, k=15, s=4)
        self.branch_long  = ResConv1D(1, 128, k=31, s=4)

        self.freq_proj = nn.Sequential(
            nn.Linear(1876, 256),
            nn.LayerNorm(256),
            nn.GELU(),
        )

        self.fuse = nn.Sequential(
            nn.Linear(128*3 + 256, d_model),
            nn.LayerNorm(d_model),
            nn.GELU(),
        )

    def forward(self, x):
        B, L, _, T_ = x.shape
        x = x.view(B*L, 1, T_)

        zs = self.branch_short(x).mean(-1)
        zm = self.branch_mid(x).mean(-1)
        zl = self.branch_long(x).mean(-1)

        # FFT in fp32 (AMP safe)
        with torch.cuda.amp.autocast(enabled=False):
            xf32 = x.squeeze(1).float()
            Xf = torch.fft.rfft(xf32, dim=-1)
            mag = torch.abs(Xf)
            mag = mag[:, :1876]
            mag = torch.log1p(mag)
            mag = mag / (mag.mean(dim=1, keepdim=True) + 1e-6)

        zf = self.freq_proj(mag)
        z = torch.cat([zs, zm, zl, zf.to(zs.dtype)], dim=-1)
        z = self.fuse(z)
        return z.view(B, L, -1)

# -------------------------
# RoPE helpers
# -------------------------
def rotate_half(x):
    x1 = x[..., ::2]
    x2 = x[..., 1::2]
    return torch.stack((-x2, x1), dim=-1).flatten(-2)

class RoPE(nn.Module):
    def __init__(self, head_dim, base=10000):
        super().__init__()
        assert head_dim % 2 == 0
        self.head_dim = head_dim
        self.base = base

    def forward(self, x):
        # x: (B, L, H, Dh)
        B, L, H, Dh = x.shape
        half = Dh // 2
        freqs = 1.0 / (self.base ** (torch.arange(half, device=x.device) / half))
        t = torch.arange(L, device=x.device)
        angles = torch.einsum("l,d->ld", t, freqs)
        cos = torch.cos(angles)[None, :, None, :]
        sin = torch.sin(angles)[None, :, None, :]
        cos = cos.repeat_interleave(2, dim=-1)
        sin = sin.repeat_interleave(2, dim=-1)
        return (x * cos) + (rotate_half(x) * sin)

# ================================================================
# V5.1 Local-Global Attention (Windowed most blocks + periodic global)
# ================================================================
def _windows(L, w):
    out = []
    s = 0
    while s < L:
        e = min(L, s + w)
        out.append((s, e))
        s = e
    return out

class MultiHeadSelfAttentionRoPE_LocalGlobal(nn.Module):
    def __init__(self, d_model=384, n_heads=8, dropout=0.1, window_size=64):
        super().__init__()
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
        self.window_size = int(window_size)

        self.qkv = nn.Linear(d_model, 3*d_model)
        self.proj = nn.Linear(d_model, d_model)
        self.drop = nn.Dropout(dropout)
        self.rope = RoPE(self.d_head)

    def forward(self, x, key_padding_mask=None, global_attn=False):
        """
        x: (B,L,D)
        key_padding_mask: (B,L) bool, True=valid
        global_attn: True => full attention, else windowed attention
        """
        B, L, D = x.shape
        qkv = self.qkv(x)
        q, k, v = qkv.chunk(3, dim=-1)

        q = q.view(B, L, self.n_heads, self.d_head)
        k = k.view(B, L, self.n_heads, self.d_head)
        v = v.view(B, L, self.n_heads, self.d_head)

        q = self.rope(q)
        k = self.rope(k)

        q = q.transpose(1, 2)  # (B,H,L,Dh)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        # ---- Global attention ----
        if global_attn or self.window_size >= L:
            scores = (q @ k.transpose(-2, -1)) / math.sqrt(self.d_head)  # (B,H,L,L)
            scores = scores.float()
            if key_padding_mask is not None:
                scores = scores.masked_fill(~key_padding_mask[:, None, None, :], -1e9)
            attn = torch.softmax(scores, dim=-1)
            attn = self.drop(attn).to(v.dtype)
            out = attn @ v
            out = out.transpose(1, 2).contiguous().view(B, L, D)
            return self.proj(out)

        # ---- Windowed attention ----
        w = self.window_size
        out = torch.zeros((B, self.n_heads, L, self.d_head), device=x.device, dtype=v.dtype)
        for (s, e) in _windows(L, w):
            qs = q[:, :, s:e, :]  # (B,H,w,Dh)
            ks = k[:, :, s:e, :]
            vs = v[:, :, s:e, :]

            scores = (qs @ ks.transpose(-2, -1)) / math.sqrt(self.d_head)  # (B,H,w,w)
            scores = scores.float()

            if key_padding_mask is not None:
                m = key_padding_mask[:, s:e]  # (B,w)
                scores = scores.masked_fill(~m[:, None, None, :], -1e9)

            attn = torch.softmax(scores, dim=-1)
            attn = self.drop(attn).to(vs.dtype)
            out[:, :, s:e, :] = attn @ vs

        out = out.transpose(1, 2).contiguous().view(B, L, D)
        return self.proj(out)

class TransformerBlockLG(nn.Module):
    def __init__(self, d_model=384, n_heads=8, drop=0.1, drop_path=0.1, window_size=64):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadSelfAttentionRoPE_LocalGlobal(d_model, n_heads, drop, window_size=window_size)
        self.ln2 = nn.LayerNorm(d_model)
        self.mlp = nn.Sequential(
            nn.Linear(d_model, 4*d_model),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(4*d_model, d_model),
        )
        self.dp = DropPath(drop_path)

    def forward(self, x, mask, global_attn=False):
        x = x + self.dp(self.attn(self.ln1(x), key_padding_mask=mask, global_attn=global_attn))
        x = x + self.dp(self.mlp(self.ln2(x)))
        return x

# -------------------------
# FINAL MODEL (V5.1)
# - same heads as V5
# - Local-Global attention in transformer
# -------------------------
class HierSleepTransformerV5_1(nn.Module):
    def __init__(
        self,
        num_classes=5,
        d_model=384,
        depth=12,          # slightly deeper than V5=10 (still cheap)
        n_heads=8,
        dur_bins=8,
        window_size=64,    # local window length
        global_every=3     # every k blocks => global attention
    ):
        super().__init__()
        self.num_classes = num_classes
        self.dur_bins = dur_bins
        self.depth = int(depth)
        self.global_every = int(global_every)

        self.encoder = EpochEncoder(d_model)
        self.blocks = nn.ModuleList([
            TransformerBlockLG(
                d_model=d_model,
                n_heads=n_heads,
                drop=0.1,
                drop_path=0.1*(i+1)/depth,
                window_size=window_size
            )
            for i in range(depth)
        ])
        self.head = nn.Linear(d_model, num_classes)

        # same as V5
        self.aux_n1 = nn.Linear(d_model, 2)
        self.aux_dur = nn.Linear(d_model, dur_bins)
        self.trans_logits = nn.Parameter(torch.zeros(num_classes, num_classes))

    def forward(self, x, mask):
        z = self.encoder(x)
        for i, blk in enumerate(self.blocks):
            use_global = (self.global_every > 0) and ((i % self.global_every) == 0)
            z = blk(z, mask, global_attn=use_global)

        main_logits = self.head(z)      # (B,L,C)
        aux_logits  = self.aux_n1(z)    # (B,L,2)
        dur_logits  = self.aux_dur(z)   # (B,L,dur_bins)
        return main_logits, aux_logits, dur_logits

# derive dur_bins from dataset bucket edges
DUR_EDGES = (2,5,10,20,40,80,160)
DUR_BINS = len(DUR_EDGES) + 1

# Recommended defaults for MESA generalization:
# - depth=12 (small boost)
# - window_size=64 (try 64 or 96)
# - global_every=3 (global attention 1/3 blocks)
model = HierSleepTransformerV5_1(
    num_classes=NUM_CLASSES,
    d_model=384,
    depth=12,
    n_heads=8,
    dur_bins=DUR_BINS,
    window_size=64,
    global_every=3
).to(device)

print("V5.1 model params (M):", sum(p.numel() for p in model.parameters()) / 1e6)
print("V5.1 Local-Global settings:", dict(depth=12, window_size=64, global_every=3))


V5.1 model params (M): 22.905512
V5.1 Local-Global settings: {'depth': 12, 'window_size': 64, 'global_every': 3}


In [9]:
# ================================================================
# V5 switches: keep V4 defaults, add V5 knobs
# ================================================================
USE_EMA = True
EMA_DECAY = 0.999

USE_AUX_N1 = True
AUX_N1_WEIGHT = 0.30

USE_LA_CE = True
LA_TAU = 1.0

USE_HARD_NEG_N1 = True
HARD_NEG_MULT = 2.0

USE_TRANS_LOSS = True
TRANS_LOSS_WEIGHT = 0.10

# V5: soft-label loss near transitions (main logits vs soft targets)
USE_SOFT_BOUNDARY_LOSS = True
SOFT_BOUNDARY_WEIGHT = 0.25   # << adjust 0.15~0.35 if needed

# V5: confusion-cost regularizer (expected cost under predicted distribution)
USE_COST_MATRIX = True
COST_WEIGHT = 0.20            # << adjust 0.10~0.30 if needed

# V5: duration-aware head (run-length bucket prediction)
USE_AUX_DUR = True
AUX_DUR_WEIGHT = 0.15          # << adjust 0.10~0.25 if needed
# extra boost on N1 epochs for duration head (N1 is transition-heavy)
AUX_DUR_N1_MULT = 1.50         # << 1.2~2.0

print("V5 switches ready.")


V5 switches ready.


In [10]:
# -------------------------
# Logit-Adjusted Cross Entropy (LA-CE)
# -------------------------
class LogitAdjustedCE(nn.Module):
    def __init__(self, class_freq, tau=1.0):
        super().__init__()
        freq = torch.tensor(class_freq, dtype=torch.float32)
        self.register_buffer("log_prior", torch.log(freq / freq.sum()))
        self.tau = float(tau)

    def forward(self, logits, targets, reduction="none"):
        logits = logits + self.tau * self.log_prior
        return F.cross_entropy(logits, targets, reduction=reduction)

# -------------------------
# Class-dependent label smoothing (vector) -- keep as V4
# -------------------------
def label_smoothing_nll(logits, targets, smooth_per_class):
    """
    logits: (N,C)
    targets: (N,)
    smooth_per_class: tensor (C,) in [0,1)
    returns per-sample loss (N,)
    """
    logp = F.log_softmax(logits, dim=-1)  # (N,C)
    nll = -logp.gather(dim=-1, index=targets.view(-1,1)).squeeze(1)  # (N,)
    smooth = -logp.mean(dim=-1)  # (N,)
    s = smooth_per_class[targets]  # (N,)
    return (1 - s) * nll + s * smooth

# V4 idea: smaller smoothing for N1
smooth_vec = torch.tensor([0.02, 0.00, 0.05, 0.05, 0.02], dtype=torch.float32).to(device)
la_ce = LogitAdjustedCE(class_freq=counts, tau=LA_TAU).to(device)

# -------------------------
# V5: boundary soft-target cross-entropy
#   loss_soft = - sum_c soft_target[c] * log_softmax(logits)[c]
# -------------------------
def soft_target_ce(logits, soft_targets):
    """
    logits: (N,C)
    soft_targets: (N,C) sum to 1
    returns per-sample loss (N,)
    """
    logp = F.log_softmax(logits, dim=-1)
    return -(soft_targets * logp).sum(dim=-1)

# -------------------------
# V5: confusion-cost matrix regularizer
#   expected_cost = sum_j p(j) * cost[y, j]
# -------------------------
def build_cost_matrix(device):
    C = NUM_CLASSES
    cost = torch.zeros((C, C), dtype=torch.float32, device=device)

    # Base: small penalty for any wrong class
    cost += 0.05
    cost.fill_diagonal_(0.0)

    # N1 critical confusions:
    # true N1 predicted W or N2 => big penalty
    cost[1, 0] = 1.00
    cost[1, 2] = 1.00
    # also penalize predicting N1 when true W or N2 (symmetry-ish)
    cost[0, 1] = 0.60
    cost[2, 1] = 0.60

    # Optional: slightly penalize N2<->N3 confusion
    cost[2, 3] = 0.20
    cost[3, 2] = 0.20

    # Optional: W<->REM confusion mild
    cost[0, 4] = 0.15
    cost[4, 0] = 0.15

    return cost

COST_MAT = build_cost_matrix(device)
print("Cost matrix ready.")


Cost matrix ready.


In [11]:
# -------------------------
# EMA (same as V4)
# -------------------------
class EMA:
    def __init__(self, model, decay=0.999):
        self.decay = float(decay)
        self.shadow = {}
        for name, p in model.named_parameters():
            if p.requires_grad:
                self.shadow[name] = p.data.detach().clone()

    @torch.no_grad()
    def update(self, model):
        for name, p in model.named_parameters():
            if name in self.shadow:
                self.shadow[name].mul_(self.decay)
                self.shadow[name].add_((1.0 - self.decay) * p.data)

    def apply(self, model):
        self.backup = {}
        for name, p in model.named_parameters():
            if name in self.shadow:
                self.backup[name] = p.data.detach().clone()
                p.data.copy_(self.shadow[name])

    def restore(self, model):
        for name, p in model.named_parameters():
            if name in self.backup:
                p.data.copy_(self.backup[name])

ema = EMA(model, EMA_DECAY) if USE_EMA else None


In [12]:
# ================================================================
# V5 Losses: keep V4 + add:
#   - soft boundary loss
#   - cost matrix reg
#   - duration aux head loss (N1-weighted)
# ================================================================

def compute_transition_loss(model, y, mask):
    """
    Supervised transition matrix training:
    minimize CE( trans_logits[y[t-1]], y[t] ) over valid adjacent pairs.
    """
    B, L = y.shape
    y_prev = y[:, :-1]
    y_next = y[:, 1:]
    m_pair = mask[:, :-1] & mask[:, 1:]

    if m_pair.sum().item() == 0:
        return torch.zeros((), device=y.device)

    y_prev_v = y_prev[m_pair]
    y_next_v = y_next[m_pair]

    trans_logits = model.trans_logits  # (C,C)
    logits_pair = trans_logits[y_prev_v]  # (N,C)
    loss = F.cross_entropy(logits_pair, y_next_v)
    return loss

def masked_loss_v5(model, main_logits, aux_logits, dur_logits, y, mask, soft_targets, dur_bucket):
    """
    main_logits: (B,L,C)
    aux_logits : (B,L,2)
    dur_logits : (B,L,DUR_BINS)
    y         : (B,L)
    mask      : (B,L) bool
    soft_targets: (B,L,C)
    dur_bucket  : (B,L)
    """
    B, L, C = main_logits.shape

    # flatten valid
    logits2 = main_logits.view(B*L, C)
    y2 = y.view(B*L)
    m2 = mask.view(B*L)

    logits_valid = logits2[m2]
    y_valid = y2[m2]

    # ---- Main loss: LA-CE + class-dependent label smoothing (same as V4) ----
    if USE_LA_CE:
        adj_logits = logits_valid + la_ce.tau * la_ce.log_prior
        loss_main_vec = label_smoothing_nll(adj_logits, y_valid, smooth_vec)
    else:
        loss_main_vec = label_smoothing_nll(logits_valid, y_valid, smooth_vec)

    # ---- V4: Hard-negative mining for N1 (when predicted W or N2) ----
    if USE_HARD_NEG_N1:
        with torch.no_grad():
            pred = torch.argmax(logits_valid, dim=-1)
            hard = (y_valid == 1) & ((pred == 0) | (pred == 2))
        loss_main_vec = loss_main_vec * torch.where(
            hard,
            torch.tensor(HARD_NEG_MULT, device=logits_valid.device),
            torch.tensor(1.0, device=logits_valid.device)
        )

    loss = loss_main_vec.mean()

    # ---- V5: Soft-label boundary loss ----
    if USE_SOFT_BOUNDARY_LOSS:
        soft2 = soft_targets.view(B*L, C)[m2]  # (N,C)
        # use same LA adjustment for soft targets for consistency
        if USE_LA_CE:
            adj_logits_soft = logits_valid + la_ce.tau * la_ce.log_prior
            loss_soft_vec = soft_target_ce(adj_logits_soft, soft2)
        else:
            loss_soft_vec = soft_target_ce(logits_valid, soft2)
        loss = loss + SOFT_BOUNDARY_WEIGHT * loss_soft_vec.mean()

    # ---- V5: Confusion-cost regularizer (expected cost) ----
    if USE_COST_MATRIX:
        probs = torch.softmax(logits_valid.float(), dim=-1)  # (N,C)
        # cost_row for each target: (N,C)
        cost_row = COST_MAT[y_valid]  # (N,C)
        expected_cost = (probs * cost_row).sum(dim=-1)  # (N,)
        loss = loss + COST_WEIGHT * expected_cost.mean()

    # ---- Aux head: N1 vs Others (same) ----
    if USE_AUX_N1:
        aux2 = aux_logits.view(B*L, 2)[m2]
        y_aux = (y_valid == 1).long()
        loss_aux = F.cross_entropy(aux2, y_aux)
        loss = loss + AUX_N1_WEIGHT * loss_aux

    # ---- V5: Duration-aware aux head ----
    if USE_AUX_DUR:
        dur2 = dur_logits.view(B*L, DUR_BINS)[m2]  # (N,DUR_BINS)
        dur_t = dur_bucket.view(B*L)[m2]           # (N,)
        # boost duration loss on N1 epochs (transition-focused)
        w = torch.ones_like(dur_t, dtype=torch.float32, device=dur2.device)
        w = w * torch.where(y_valid == 1, torch.tensor(AUX_DUR_N1_MULT, device=dur2.device), torch.tensor(1.0, device=dur2.device))
        loss_dur = F.cross_entropy(dur2, dur_t, reduction="none")
        loss_dur = (loss_dur * w).mean()
        loss = loss + AUX_DUR_WEIGHT * loss_dur

    # ---- Transition loss (same) ----
    if USE_TRANS_LOSS:
        loss_trans = compute_transition_loss(model, y, mask)
        loss = loss + TRANS_LOSS_WEIGHT * loss_trans

    return loss


In [13]:
# -------------------------
# Optimizer + OneCycle (same values as V4)
# -------------------------
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-2)

EPOCHS = 60
steps_per_epoch = len(train_seq_loader)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=5e-4,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    pct_start=0.15,
    div_factor=20.0,
    final_div_factor=100.0
)
scheduler_is_step_per_batch = True
print("Using OneCycleLR (max_lr=5e-4)")

scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))


Using OneCycleLR (max_lr=5e-4)


In [14]:
# -------------------------
# Metrics helpers (same as V4)
# -------------------------
def _ece_from_probs(y_true, probs, n_bins=15):
    conf = probs.max(axis=1)
    pred = probs.argmax(axis=1)
    acc = (pred == y_true).astype(np.float32)

    bins = np.linspace(0.0, 1.0, n_bins+1)
    ece = 0.0
    for i in range(n_bins):
        lo, hi = bins[i], bins[i+1]
        m = (conf >= lo) & (conf < hi)
        if m.sum() == 0:
            continue
        bin_acc = acc[m].mean()
        bin_conf = conf[m].mean()
        ece += (m.mean()) * abs(bin_acc - bin_conf)
    return float(ece)

def _auroc_auprc_multiclass(y_true, probs, num_classes=5):
    Y = label_binarize(y_true, classes=list(range(num_classes)))
    aurocs, auprcs = [], []
    for c in range(num_classes):
        if Y[:, c].sum() == 0:
            continue
        try:
            aurocs.append(roc_auc_score(Y[:, c], probs[:, c]))
            auprcs.append(average_precision_score(Y[:, c], probs[:, c]))
        except Exception:
            pass
    if len(aurocs) == 0:
        return float("nan"), float("nan")
    return float(np.mean(aurocs)), float(np.mean(auprcs))

def estimate_transition_from_train(df, num_classes=5, eps=1.0):
    Tm = np.zeros((num_classes, num_classes), dtype=np.float64)
    for p in tqdm(df["npz_path"].tolist(), desc="Estimating transitions(train)", leave=False):
        d = np.load(p, allow_pickle=True)
        y = d["y"].astype(np.int64)
        y = y[y >= 0]
        if len(y) < 2:
            continue
        a = y[:-1]; b = y[1:]
        for i, j in zip(a, b):
            if 0 <= i < num_classes and 0 <= j < num_classes:
                Tm[i, j] += 1.0
    Tm = Tm + eps
    Tm = Tm / Tm.sum(axis=1, keepdims=True)
    return Tm.astype(np.float32)

def viterbi_decode(log_emis, log_trans):
    L, C = log_emis.shape
    dp = np.zeros((L, C), dtype=np.float32)
    back = np.zeros((L, C), dtype=np.int32)
    dp[0] = log_emis[0]
    for t in range(1, L):
        scores = dp[t-1][:, None] + log_trans
        back[t] = np.argmax(scores, axis=0)
        dp[t] = log_emis[t] + scores[back[t], np.arange(C)]
    path = np.zeros((L,), dtype=np.int32)
    path[-1] = int(np.argmax(dp[-1]))
    for t in range(L-2, -1, -1):
        path[t] = back[t+1, path[t+1]]
    return path

# V4 learned smoothing at inference (keep)
USE_LEARNED_SMOOTHING = True
def apply_learned_smoothing_probs(probs, model):
    Tm = torch.softmax(model.trans_logits, dim=1)  # (C,C)
    return probs @ Tm

# Viterbi option (keep off)
USE_VITERBI = False
Tmat = estimate_transition_from_train(df_train, num_classes=NUM_CLASSES, eps=1.0) if USE_VITERBI else None
logT = np.log(Tmat + 1e-12).astype(np.float32) if USE_VITERBI else None

def tensor_stats(name, t):
    t = t.detach()
    finite = torch.isfinite(t).all().item()
    absmax = t.abs().max().item() if t.numel() else 0.0
    return f"{name}: finite={finite} absmax={absmax:.6g} dtype={t.dtype} shape={tuple(t.shape)}"

@torch.no_grad()
def check_model_params_finite(model):
    for n, p in model.named_parameters():
        if p is None:
            continue
        if not torch.isfinite(p).all():
            return False, n
    return True, None


In [15]:
# -------------------------
# Eval (updated to accept new batch outputs, still reports same metrics)
# -------------------------
@torch.no_grad()
def eval_sequence(model, loader, desc="Eval"):
    model.eval()

    all_true, all_pred = [], []
    all_probs = []

    all_true_v, all_pred_v = [], []

    total_loss = 0.0
    total_n = 0
    bad_batches = 0
    first_bad_printed = False

    for bidx, (xb, yb, mb, sb, db) in enumerate(tqdm(loader, desc=desc, leave=False)):
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        mb = mb.to(device, non_blocking=True)
        sb = sb.to(device, non_blocking=True)
        db = db.to(device, non_blocking=True)

        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            main_logits, aux_logits, dur_logits = model(xb, mb)
            loss = masked_loss_v5(model, main_logits, aux_logits, dur_logits, yb, mb, sb, db)

        if (not torch.isfinite(main_logits).all()) or (not torch.isfinite(loss)):
            bad_batches += 1
            if not first_bad_printed:
                first_bad_printed = True
                print(f"\n[WARN] {desc}: non-finite logits/loss at batch={bidx}")
                print(tensor_stats("logits", main_logits))
                print("loss:", float(loss.item()) if torch.isfinite(loss) else loss)
            continue

        probs = torch.softmax(main_logits.float(), dim=-1)

        if USE_LEARNED_SMOOTHING:
            probs = apply_learned_smoothing_probs(probs, model)

        if not torch.isfinite(probs).all():
            bad_batches += 1
            if not first_bad_printed:
                first_bad_printed = True
                print(f"\n[WARN] {desc}: non-finite probs at batch={bidx}")
                print(tensor_stats("logits", main_logits))
                print(tensor_stats("probs", probs))
            continue

        pred = torch.argmax(probs, dim=-1)

        yv = yb[mb].detach().cpu().numpy()
        pv = pred[mb].detach().cpu().numpy()
        pr = probs[mb].detach().cpu().numpy()

        if yv.size == 0:
            continue

        all_true.append(yv)
        all_pred.append(pv)
        all_probs.append(pr)

        if USE_VITERBI:
            probs_np = probs.detach().cpu().numpy()
            y_np = yb.detach().cpu().numpy()
            m_np = mb.detach().cpu().numpy()
            for i in range(probs_np.shape[0]):
                Li = int(m_np[i].sum())
                if Li <= 0:
                    continue
                emis = probs_np[i, :Li]
                yseq = y_np[i, :Li].astype(np.int64)
                if not np.isfinite(emis).all():
                    continue
                path = viterbi_decode(np.log(emis + 1e-12).astype(np.float32), logT)
                all_true_v.append(yseq)
                all_pred_v.append(path.astype(np.int64))

        n = int(mb.sum().item())
        total_loss += float(loss.item()) * n
        total_n += n

    if len(all_true) == 0:
        print(f"\n[STOP] {desc}: 0 valid batches collected (bad_batches={bad_batches}).")
        return {
            "loss": float("nan"),
            "acc": float("nan"),
            "macro_f1": float("nan"),
            "kappa": float("nan"),
            "AUROC": float("nan"),
            "AUPRC": float("nan"),
            "meanConf": float("nan"),
            "ECE": float("nan"),
            "f1_per_class": {LABELS[i]: float("nan") for i in range(NUM_CLASSES)},
            "cm": np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=np.int64),
            "bad_batches": int(bad_batches),
        }

    y_true = np.concatenate(all_true)
    y_pred = np.concatenate(all_pred)
    probs_all = np.concatenate(all_probs)

    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average="macro")
    kappa = cohen_kappa_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=list(range(NUM_CLASSES)))

    auroc, auprc = _auroc_auprc_multiclass(y_true, probs_all, num_classes=NUM_CLASSES)
    mean_conf = float(probs_all.max(axis=1).mean())
    ece = _ece_from_probs(y_true, probs_all, n_bins=15)

    f1_per = {}
    for i in range(NUM_CLASSES):
        f1_per[LABELS[i]] = float(f1_score((y_true==i).astype(int), (y_pred==i).astype(int)))

    out = {
        "loss": total_loss / max(total_n, 1),
        "acc": float(acc),
        "macro_f1": float(mf1),
        "kappa": float(kappa),
        "AUROC": auroc,
        "AUPRC": auprc,
        "meanConf": mean_conf,
        "ECE": ece,
        "f1_per_class": f1_per,
        "cm": cm,
        "bad_batches": int(bad_batches),
    }

    if USE_VITERBI and len(all_true_v) > 0:
        yt = np.concatenate(all_true_v)
        yp = np.concatenate(all_pred_v)
        out["viterbi_acc"] = float(accuracy_score(yt, yp))
        out["viterbi_macro_f1"] = float(f1_score(yt, yp, average="macro"))
        out["viterbi_kappa"] = float(cohen_kappa_score(yt, yp))
        out["viterbi_cm"] = confusion_matrix(yt, yp, labels=list(range(NUM_CLASSES)))

    return out


In [16]:
# -------------------------
# Train one epoch (updated batch tuple + V5 loss)
# -------------------------
def train_one_epoch(model, loader, epoch, grad_clip=1.0, droppath_target=0.10, droppath_warm_epochs=10):
    model.train()
    running_loss = 0.0
    n_seen = 0

    # DropPath warmup (keep)
    warm = min(1.0, epoch / max(1, droppath_warm_epochs))
    for blk in model.blocks:
        blk.dp.drop_prob = droppath_target * warm

    pbar = tqdm(loader, desc=f"Train epoch {epoch}", leave=False)
    for step, (xb, yb, mb, sb, db) in enumerate(pbar):
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        mb = mb.to(device, non_blocking=True)
        sb = sb.to(device, non_blocking=True)
        db = db.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            main_logits, aux_logits, dur_logits = model(xb, mb)
            loss = masked_loss_v5(model, main_logits, aux_logits, dur_logits, yb, mb, sb, db)

        if not torch.isfinite(loss):
            print("\n[STOP] Non-finite loss detected:", float(loss.item()))
            print(tensor_stats("logits", main_logits))
            ok, bad = check_model_params_finite(model)
            print("params finite:", ok, "| first bad:", bad)
            return float("nan")

        scaler.scale(loss).backward()

        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

        scaler.step(optimizer)
        scaler.update()

        if scheduler_is_step_per_batch:
            scheduler.step()

        if USE_EMA:
            ema.update(model)

        n = int(mb.sum().item())
        running_loss += float(loss.item()) * n
        n_seen += n
        pbar.set_postfix(loss=running_loss/max(n_seen,1), lr=optimizer.param_groups[0]["lr"])

    return running_loss / max(n_seen, 1)


In [17]:
# %% [code]
# -------------------------
# Training loop + checkpoints
# - Save BEST by VAL macro-F1 (main)
# - Save BEST by MESA macro-F1 (secondary; if MESA loader exists)
# - Keep Top-K by VAL macro-F1 for ensemble
# -------------------------
CKPT_DIR = ROOT / "checkpoints_hier_rope_seq_v5_1"   # <= new folder to avoid overwriting old runs
CKPT_DIR.mkdir(parents=True, exist_ok=True)

best_val_path  = CKPT_DIR / "BEST_VAL_macroF1.pt"
best_mesa_path = CKPT_DIR / "BEST_MESA_macroF1.pt"

best_val  = -1.0
best_mesa = -1.0

topk_paths = []
ENSEMBLE_K = 5

def _make_payload(epoch, tr_loss, val_m, test_m, ext_m, mesa_m):
    payload = {
        "epoch": epoch,
        "train_loss": float(tr_loss),

        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),

        "val_metrics": val_m,
        "test_metrics": test_m,
        "ext_metrics": ext_m,
        "mesa_metrics": mesa_m,

        "class_weights": class_weights.detach().cpu().numpy(),

        "use_ema": USE_EMA,
        "ema_decay": EMA_DECAY,
        "ema_shadow": {k: v.detach().cpu() for k, v in ema.shadow.items()} if USE_EMA else None,

        "use_learned_smoothing": USE_LEARNED_SMOOTHING,
        "use_viterbi": USE_VITERBI,
        "Tmat": Tmat if USE_VITERBI else None,

        # V5 config snapshot (works for V5.1 too)
        "v5_soft_boundary": dict(enabled=USE_SOFT_BOUNDARY_LOSS, weight=SOFT_BOUNDARY_WEIGHT),
        "v5_cost_matrix": dict(enabled=USE_COST_MATRIX, weight=COST_WEIGHT),
        "v5_aux_dur": dict(enabled=USE_AUX_DUR, weight=AUX_DUR_WEIGHT, n1_mult=AUX_DUR_N1_MULT, dur_edges=DUR_EDGES),
    }
    return payload


for epoch in range(1, EPOCHS + 1):
    tr_loss = train_one_epoch(model, train_seq_loader, epoch)

    # Evaluate with EMA weights
    if USE_EMA:
        ema.apply(model)

    val_m  = eval_sequence(model, val_seq_loader,  desc="VAL")
    test_m = eval_sequence(model, test_seq_loader, desc="SHHS1 TEST")
    ext_m  = eval_sequence(model, ext_seq_loader,  desc="SHHS2 EXT")
    mesa_m = eval_sequence(model, mesa_seq_loader, desc="MESA EXT") if mesa_seq_loader is not None else None

    if USE_EMA:
        ema.restore(model)

    print(f"\nEpoch {epoch:02d} | train_loss={tr_loss:.4f}")

    print(f"  VAL   : loss={val_m['loss']:.4f} acc={val_m['acc']:.4f} macroF1={val_m['macro_f1']:.4f} "
          f"kappa={val_m['kappa']:.4f} AUROC={val_m['AUROC']:.4f} AUPRC={val_m['AUPRC']:.4f} "
          f"meanConf={val_m['meanConf']:.4f} ECE={val_m['ECE']:.4f}")
    print(f"  F1/class: {val_m['f1_per_class']}")

    print(f"  TEST1 : loss={test_m['loss']:.4f} acc={test_m['acc']:.4f} macroF1={test_m['macro_f1']:.4f} "
          f"kappa={test_m['kappa']:.4f} AUROC={test_m['AUROC']:.4f} AUPRC={test_m['AUPRC']:.4f} "
          f"meanConf={test_m['meanConf']:.4f} ECE={test_m['ECE']:.4f}")

    print(f"  SHHS2 : loss={ext_m['loss']:.4f} acc={ext_m['acc']:.4f} macroF1={ext_m['macro_f1']:.4f} "
          f"kappa={ext_m['kappa']:.4f} AUROC={ext_m['AUROC']:.4f} AUPRC={ext_m['AUPRC']:.4f} "
          f"meanConf={ext_m['meanConf']:.4f} ECE={ext_m['ECE']:.4f}")

    if mesa_m is not None:
        print(f"  MESA  : loss={mesa_m['loss']:.4f} acc={mesa_m['acc']:.4f} macroF1={mesa_m['macro_f1']:.4f} "
              f"kappa={mesa_m['kappa']:.4f} AUROC={mesa_m['AUROC']:.4f} AUPRC={mesa_m['AUPRC']:.4f} "
              f"meanConf={mesa_m['meanConf']:.4f} ECE={mesa_m['ECE']:.4f}")
        print(f"  MESA F1/class: {mesa_m['f1_per_class']}")

    if USE_VITERBI:
        print(f"  VAL(viterbi): acc={val_m['viterbi_acc']:.4f} macroF1={val_m['viterbi_macro_f1']:.4f} kappa={val_m['viterbi_kappa']:.4f}")

    payload = _make_payload(epoch, tr_loss, val_m, test_m, ext_m, mesa_m)

    # ============================================================
    # (A) Save BEST by VAL macro-F1 (main)
    # ============================================================
    if (val_m is not None) and (val_m.get("macro_f1", float("nan")) == val_m.get("macro_f1", float("nan"))):
        if val_m["macro_f1"] > best_val:
            best_val = float(val_m["macro_f1"])
            payload["best_val_macroF1"] = best_val
            torch.save(payload, best_val_path)
            print("  ? Saved BEST_VAL:", best_val_path.name, f"(val_macroF1={best_val:.4f})")

            ck = CKPT_DIR / f"VALBEST_ep{epoch:03d}_valF1_{best_val:.4f}.pt"
            torch.save(payload, ck)
            topk_paths.append(ck)
            topk_paths = topk_paths[-ENSEMBLE_K:]
            print("  ? Added to VAL Top-K:", ck.name)

    # ============================================================
    # (B) Save BEST by MESA macro-F1 (secondary)
    # ============================================================
    if mesa_m is not None:
        mf1 = mesa_m.get("macro_f1", float("nan"))
        if mf1 == mf1:  # not NaN
            if mf1 > best_mesa:
                best_mesa = float(mf1)
                payload["best_mesa_macroF1"] = best_mesa
                torch.save(payload, best_mesa_path)
                print("  ? Saved BEST_MESA:", best_mesa_path.name, f"(mesa_macroF1={best_mesa:.4f})")

                ck2 = CKPT_DIR / f"MESABEST_ep{epoch:03d}_mesaF1_{best_mesa:.4f}.pt"
                torch.save(payload, ck2)
                print("  ? Snapshot MESA best:", ck2.name)

print("\n==============================")
print("Training finished")
print("==============================")
print("BEST VAL macroF1 :", f"{best_val:.4f}", "|", best_val_path)
if mesa_seq_loader is not None:
    print("BEST MESA macroF1:", f"{best_mesa:.4f}", "|", best_mesa_path)

print("\nTop-K VAL checkpoints for ensemble:")
for p in topk_paths:
    print(" -", p)


Train epoch 1:   0%|                                   | 0/2190 [00:00<?, ?it/s]/usr/local/anaconda3/envs/akbar1/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)



Epoch 01 | train_loss=1.3762
  VAL   : loss=1.2399 acc=0.7872 macroF1=0.6513 kappa=0.6938 AUROC=0.9473 AUPRC=0.7838 meanConf=0.2081 ECE=0.5791
  F1/class: {'W': 0.8225161576933548, 'N1': 0.05058552087414566, 'N2': 0.8033134343332999, 'N3': 0.7542364701826956, 'REM': 0.8258525709836858}
  TEST1 : loss=1.2185 acc=0.7935 macroF1=0.6554 kappa=0.7028 AUROC=0.9509 AUPRC=0.7917 meanConf=0.2081 ECE=0.5855
  SHHS2 : loss=1.2955 acc=0.7675 macroF1=0.6282 kappa=0.6688 AUROC=0.9456 AUPRC=0.7761 meanConf=0.2077 ECE=0.5598
  MESA  : loss=1.7274 acc=0.6964 macroF1=0.5758 kappa=0.5577 AUROC=0.8980 AUPRC=0.6962 meanConf=0.2069 ECE=0.4895
  MESA F1/class: {'W': 0.7600170124661046, 'N1': 0.0027893560708116077, 'N2': 0.7053987029187828, 'N3': 0.6437907217558705, 'REM': 0.7672267376707401}
  ? Saved BEST_VAL: BEST_VAL_macroF1.pt (val_macroF1=0.6513)
  ? Added to VAL Top-K: VALBEST_ep001_valF1_0.6513.pt
  ? Saved BEST_MESA: BEST_MESA_macroF1.pt (mesa_macroF1=0.5758)
  ? Snapshot MESA best: MESABEST_ep001_m


Epoch 02 | train_loss=1.1840
  VAL   : loss=1.0878 acc=0.8294 macroF1=0.7371 kappa=0.7574 AUROC=0.9583 AUPRC=0.8177 meanConf=0.2318 ECE=0.5976
  F1/class: {'W': 0.8929749536464091, 'N1': 0.32192874692874696, 'N2': 0.8415611380865352, 'N3': 0.7665224441816495, 'REM': 0.8625898171277022}
  TEST1 : loss=1.0563 acc=0.8376 macroF1=0.7423 kappa=0.7687 AUROC=0.9620 AUPRC=0.8281 meanConf=0.2318 ECE=0.6058
  SHHS2 : loss=1.0430 acc=0.8366 macroF1=0.7236 kappa=0.7678 AUROC=0.9621 AUPRC=0.8265 meanConf=0.2308 ECE=0.6058
  MESA  : loss=1.5583 acc=0.7224 macroF1=0.6057 kappa=0.5956 AUROC=0.9181 AUPRC=0.7332 meanConf=0.2270 ECE=0.4954
  MESA F1/class: {'W': 0.7874290416156475, 'N1': 0.1026176631099569, 'N2': 0.7370577437978761, 'N3': 0.610963880057253, 'REM': 0.7902263250543472}
  ? Saved BEST_VAL: BEST_VAL_macroF1.pt (val_macroF1=0.7371)
  ? Added to VAL Top-K: VALBEST_ep002_valF1_0.7371.pt
  ? Saved BEST_MESA: BEST_MESA_macroF1.pt (mesa_macroF1=0.6057)
  ? Snapshot MESA best: MESABEST_ep002_mesaF


Epoch 03 | train_loss=1.1380
  VAL   : loss=1.0352 acc=0.8370 macroF1=0.7680 kappa=0.7721 AUROC=0.9611 AUPRC=0.8254 meanConf=0.2797 ECE=0.5573
  F1/class: {'W': 0.8984920431490275, 'N1': 0.44173441734417346, 'N2': 0.8473815833169547, 'N3': 0.7813822284908323, 'REM': 0.8711978877327458}
  TEST1 : loss=1.0014 acc=0.8450 macroF1=0.7741 kappa=0.7831 AUROC=0.9648 AUPRC=0.8363 meanConf=0.2798 ECE=0.5652
  SHHS2 : loss=0.9763 acc=0.8460 macroF1=0.7592 kappa=0.7843 AUROC=0.9653 AUPRC=0.8356 meanConf=0.2782 ECE=0.5678
  MESA  : loss=1.5273 acc=0.7462 macroF1=0.6558 kappa=0.6351 AUROC=0.9203 AUPRC=0.7437 meanConf=0.2696 ECE=0.4766
  MESA F1/class: {'W': 0.8132645434160393, 'N1': 0.2525295360238428, 'N2': 0.7573898988588511, 'N3': 0.6503484813853286, 'REM': 0.805698821154993}
  ? Saved BEST_VAL: BEST_VAL_macroF1.pt (val_macroF1=0.7680)
  ? Added to VAL Top-K: VALBEST_ep003_valF1_0.7680.pt
  ? Saved BEST_MESA: BEST_MESA_macroF1.pt (mesa_macroF1=0.6558)
  ? Snapshot MESA best: MESABEST_ep003_mesaF


Epoch 04 | train_loss=1.1066
  VAL   : loss=0.9896 acc=0.8369 macroF1=0.7786 kappa=0.7746 AUROC=0.9630 AUPRC=0.8283 meanConf=0.3612 ECE=0.4756
  F1/class: {'W': 0.9020108660360613, 'N1': 0.4918646636651362, 'N2': 0.8445191068778235, 'N3': 0.7820647696305216, 'REM': 0.8724920607567515}
  TEST1 : loss=0.9558 acc=0.8449 macroF1=0.7852 kappa=0.7856 AUROC=0.9665 AUPRC=0.8388 meanConf=0.3614 ECE=0.4836
  SHHS2 : loss=0.9286 acc=0.8479 macroF1=0.7742 kappa=0.7891 AUROC=0.9673 AUPRC=0.8408 meanConf=0.3577 ECE=0.4902
  MESA  : loss=1.5120 acc=0.7448 macroF1=0.6647 kappa=0.6369 AUROC=0.9198 AUPRC=0.7425 meanConf=0.3396 ECE=0.4053
  MESA F1/class: {'W': 0.8105911841460651, 'N1': 0.30588381231051004, 'N2': 0.7568977574893391, 'N3': 0.6500011293929352, 'REM': 0.8000120698132196}
  ? Saved BEST_VAL: BEST_VAL_macroF1.pt (val_macroF1=0.7786)
  ? Added to VAL Top-K: VALBEST_ep004_valF1_0.7786.pt
  ? Saved BEST_MESA: BEST_MESA_macroF1.pt (mesa_macroF1=0.6647)
  ? Snapshot MESA best: MESABEST_ep004_mesa

...


Epoch 56 | train_loss=0.7345
  VAL   : loss=0.8967 acc=0.8544 macroF1=0.7974 kappa=0.7990 AUROC=0.9661 AUPRC=0.8373 meanConf=0.7199 ECE=0.1344
  F1/class: {'W': 0.9113338737973603, 'N1': 0.5052182068525134, 'N2': 0.8647356835889731, 'N3': 0.8263716918792161, 'REM': 0.8794089430995601}
  TEST1 : loss=0.8473 acc=0.8645 macroF1=0.8066 kappa=0.8125 AUROC=0.9704 AUPRC=0.8524 meanConf=0.7196 ECE=0.1449
  SHHS2 : loss=0.8261 acc=0.8676 macroF1=0.7971 kappa=0.8163 AUROC=0.9678 AUPRC=0.8450 meanConf=0.7184 ECE=0.1492
  MESA  : loss=1.6370 acc=0.7604 macroF1=0.6179 kappa=0.6561 AUROC=0.8973 AUPRC=0.6543 meanConf=0.6980 ECE=0.0624
  MESA F1/class: {'W': 0.8801862803956113, 'N1': 0.36964501276258516, 'N2': 0.775996612285542, 'N3': 0.2578585894162191, 'REM': 0.8059122093898531}



Epoch 57 | train_loss=0.7317
  VAL   : loss=0.8965 acc=0.8544 macroF1=0.7975 kappa=0.7990 AUROC=0.9661 AUPRC=0.8374 meanConf=0.7200 ECE=0.1344
  F1/class: {'W': 0.9113929477944708, 'N1': 0.5059710747934628, 'N2': 0.8647055259613456, 'N3': 0.8264935405600368, 'REM': 0.8788160149556005}
  TEST1 : loss=0.8474 acc=0.8643 macroF1=0.8064 kappa=0.8122 AUROC=0.9703 AUPRC=0.8523 meanConf=0.7197 ECE=0.1446
  SHHS2 : loss=0.8274 acc=0.8674 macroF1=0.7969 kappa=0.8160 AUROC=0.9676 AUPRC=0.8446 meanConf=0.7186 ECE=0.1488
  MESA  : loss=1.6385 acc=0.7594 macroF1=0.6188 kappa=0.6552 AUROC=0.8974 AUPRC=0.6547 meanConf=0.6985 ECE=0.0609
  MESA F1/class: {'W': 0.8789030321083758, 'N1': 0.37231641511385277, 'N2': 0.7756851238358365, 'N3': 0.26282667408918325, 'REM': 0.8040424204616344}



Epoch 58 | train_loss=0.7312
  VAL   : loss=0.8971 acc=0.8543 macroF1=0.7975 kappa=0.7990 AUROC=0.9660 AUPRC=0.8374 meanConf=0.7205 ECE=0.1339
  F1/class: {'W': 0.9114835303746428, 'N1': 0.5070025188916877, 'N2': 0.8644828124599471, 'N3': 0.8256660658692446, 'REM': 0.8788082520947721}
  TEST1 : loss=0.8477 acc=0.8644 macroF1=0.8065 kappa=0.8124 AUROC=0.9703 AUPRC=0.8525 meanConf=0.7201 ECE=0.1443
  SHHS2 : loss=0.8288 acc=0.8673 macroF1=0.7968 kappa=0.8159 AUROC=0.9675 AUPRC=0.8445 meanConf=0.7191 ECE=0.1481
  MESA  : loss=1.6617 acc=0.7556 macroF1=0.6136 kappa=0.6500 AUROC=0.8948 AUPRC=0.6494 meanConf=0.6985 ECE=0.0571
  MESA F1/class: {'W': 0.874987230825808, 'N1': 0.366399392380676, 'N2': 0.7740154561004935, 'N3': 0.2549783010890235, 'REM': 0.7976458546571136}



Epoch 59 | train_loss=0.7309
  VAL   : loss=0.8973 acc=0.8538 macroF1=0.7970 kappa=0.7984 AUROC=0.9660 AUPRC=0.8371 meanConf=0.7199 ECE=0.1339
  F1/class: {'W': 0.9113124927626979, 'N1': 0.5052021967049426, 'N2': 0.8638882534465614, 'N3': 0.8258085614310475, 'REM': 0.8787680902538079}
  TEST1 : loss=0.8478 acc=0.8640 macroF1=0.8063 kappa=0.8119 AUROC=0.9702 AUPRC=0.8521 meanConf=0.7195 ECE=0.1445
  SHHS2 : loss=0.8282 acc=0.8668 macroF1=0.7964 kappa=0.8153 AUROC=0.9676 AUPRC=0.8444 meanConf=0.7184 ECE=0.1484
  MESA  : loss=1.6545 acc=0.7562 macroF1=0.6152 kappa=0.6509 AUROC=0.8955 AUPRC=0.6514 meanConf=0.6982 ECE=0.0580
  MESA F1/class: {'W': 0.8754429959736342, 'N1': 0.36845183953350913, 'N2': 0.7737054114426044, 'N3': 0.25983416682903593, 'REM': 0.7986418076305717}



Epoch 60 | train_loss=0.7269
  VAL   : loss=0.8973 acc=0.8542 macroF1=0.7974 kappa=0.7988 AUROC=0.9661 AUPRC=0.8373 meanConf=0.7203 ECE=0.1339
  F1/class: {'W': 0.9114018435272994, 'N1': 0.50579814459373, 'N2': 0.8643699493124304, 'N3': 0.8260416051642361, 'REM': 0.879179170454121}
  TEST1 : loss=0.8480 acc=0.8642 macroF1=0.8063 kappa=0.8121 AUROC=0.9703 AUPRC=0.8523 meanConf=0.7200 ECE=0.1442
  SHHS2 : loss=0.8272 acc=0.8675 macroF1=0.7969 kappa=0.8161 AUROC=0.9676 AUPRC=0.8446 meanConf=0.7189 ECE=0.1486
  MESA  : loss=1.6603 acc=0.7564 macroF1=0.6138 kappa=0.6510 AUROC=0.8951 AUPRC=0.6499 meanConf=0.6988 ECE=0.0576
  MESA F1/class: {'W': 0.8754923565608126, 'N1': 0.36817944384698115, 'N2': 0.7744424761581208, 'N3': 0.252421341822384, 'REM': 0.7983495556495981}

Training finished
BEST VAL macroF1 : 0.8019 | /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/checkpoints_hier_rope_seq_v5_1/BEST_VAL_macroF1.pt
BEST MESA macroF1: 0.7011 | /data2/Akbar1/sleep_stages_Dibatic/shhs_s

In [1]:
# =========================
# CELL 0: Imports (safe)
# =========================
import os
import re
import json
from pathlib import Path

import numpy as np
import torch


In [2]:
# ============================================
# CELL 1: Paths (EDIT ONLY THIS CELL IF NEEDED)
# ============================================
ROOT = Path("/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/")

# Your training log shows checkpoints in:
# /data2/Akbar1/.../checkpoints_hier_rope_seq_v5_1/
# so default to that (change if needed).
CKPT_DIR = ROOT / "checkpoints_hier_rope_seq_v5_1"

# If you want to force a specific file, set BEST_CKPT explicitly.
# Otherwise it will auto-pick BEST_VAL_macroF1.pt if exists.
BEST_CKPT = CKPT_DIR / "BEST_VAL_macroF1.pt"

assert CKPT_DIR.exists(), f"Checkpoint folder not found: {CKPT_DIR}"

if not BEST_CKPT.exists():
    # fallback: search common best names
    candidates = [
        CKPT_DIR / "BEST_VAL_macroF1.pt",
        CKPT_DIR / "V5_best_val_macroF1.pt",
        CKPT_DIR / "best.pt",
        CKPT_DIR / "checkpoint_best.pt",
    ]
    found = None
    for c in candidates:
        if c.exists():
            found = c
            break
    if found is None:
        # last fallback: pick newest .pt
        pts = sorted(CKPT_DIR.glob("*.pt"), key=lambda p: p.stat().st_mtime, reverse=True)
        assert len(pts) > 0, f"No .pt checkpoints found in: {CKPT_DIR}"
        found = pts[0]
    BEST_CKPT = found

print("Using checkpoint:", BEST_CKPT)


Using checkpoint: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/checkpoints_hier_rope_seq_v5_1/BEST_VAL_macroF1.pt


In [3]:
# ============================================
# CELL 2: Load checkpoint (robust)
# ============================================
ckpt = torch.load(BEST_CKPT, map_location="cpu")

print("Checkpoint keys:", sorted(list(ckpt.keys()))[:40], "...")
print("Has model_state:", "model_state" in ckpt)
print("Has state_dict:", "state_dict" in ckpt)
print("Has model object:", "model" in ckpt)
print("Has ema_shadow:", "ema_shadow" in ckpt)
print("use_ema flag:", ckpt.get("use_ema", False))
print("arch/model_class:", ckpt.get("arch", None), ckpt.get("model_class", None))


Checkpoint keys: ['Tmat', 'best_val_macroF1', 'class_weights', 'ema_decay', 'ema_shadow', 'epoch', 'ext_metrics', 'mesa_metrics', 'model_state', 'optimizer_state', 'test_metrics', 'train_loss', 'use_ema', 'use_learned_smoothing', 'use_viterbi', 'v5_aux_dur', 'v5_cost_matrix', 'v5_soft_boundary', 'val_metrics'] ...
Has model_state: True
Has state_dict: False
Has model object: False
Has ema_shadow: True
use_ema flag: True
arch/model_class: None None


In [4]:
# ============================================================
# CELL 3: Build / recover model WITHOUT hardcoding class name
# ============================================================

def _find_candidate_model_classes(globs: dict):
    """
    Heuristic: find classes that likely represent your model.
    Priority: names containing V5, Sleep, Transformer, Hier, etc.
    """
    candidates = []
    for name, obj in globs.items():
        if isinstance(obj, type):  # is a class
            n = name.lower()
            if any(k in n for k in ["sleep", "transformer", "hier", "stage", "v5"]):
                candidates.append(name)

    def score(nm: str):
        n = nm.lower()
        s = 0
        if "v5" in n: s += 10
        if "hier" in n: s += 6
        if "sleep" in n: s += 6
        if "transformer" in n: s += 6
        if "net" in n: s += 2
        return s

    candidates = sorted(set(candidates), key=lambda x: score(x), reverse=True)
    return candidates

def _build_model_from_notebook_or_ckpt(ckpt: dict):
    """
    Build model using (in order):
      A) ckpt["model"] object
      B) build_model()/make_model()/get_model() if exists
      C) ckpt["model_class"]/ckpt["arch"] class name
      D) autodetect from globals
    """
    # A) checkpoint contains full model object
    if "model" in ckpt and ckpt["model"] is not None:
        m = ckpt["model"]
        if isinstance(m, torch.nn.Module):
            print("[Model] Using ckpt['model'] object directly.")
            return m
        else:
            print("[Model] ckpt['model'] exists but is not a torch.nn.Module. Ignoring.")

    # B) common builder functions in notebook
    for fn_name in ["build_model", "make_model", "get_model"]:
        fn = globals().get(fn_name, None)
        if callable(fn):
            try:
                print(f"[Model] Using notebook builder: {fn_name}()")
                return fn()
            except TypeError:
                # some builders require args; fall through
                print(f"[Model] Found {fn_name} but it needs args. Will try other methods.")
            except Exception as e:
                print(f"[Model] {fn_name}() failed: {e}. Will try other methods.")

    # C) checkpoint tells class name
    for key in ["model_class", "arch"]:
        cls_name = ckpt.get(key, None)
        if isinstance(cls_name, str) and cls_name in globals() and isinstance(globals()[cls_name], type):
            cls = globals()[cls_name]
            print(f"[Model] Using class from ckpt['{key}']:", cls_name)
            # Try instantiate with common patterns
            # NOTE: adjust here if your constructor is very custom
            try:
                return cls(num_classes=NUM_CLASSES).to(device)
            except Exception:
                try:
                    return cls(NUM_CLASSES).to(device)
                except Exception:
                    # last attempt: no args
                    return cls().to(device)

    # D) auto-detect likely class names in notebook globals
    candidates = _find_candidate_model_classes(globals())
    if len(candidates) == 0:
        raise NameError(
            "No candidate model class found in notebook globals.\n"
            "Make sure you executed the cell that defines your model class (V5) before running evaluation."
        )

    print("[Model] Auto-detected candidate classes (top 10):", candidates[:10])
    picked = candidates[0]
    cls = globals()[picked]
    print("[Model] Picking:", picked)

    # Try instantiate with your earlier hyperparams (edit ONLY if needed)
    # These are safe tries; if fails, it will attempt simpler calls.
    try:
        return cls(num_classes=NUM_CLASSES, d_model=384, depth=10, n_heads=8).to(device)
    except Exception:
        try:
            return cls(num_classes=NUM_CLASSES).to(device)
        except Exception:
            try:
                return cls(NUM_CLASSES).to(device)
            except Exception:
                return cls().to(device)

# ---- build model now ----
assert "device" in globals(), "device not found. Make sure you defined: device = torch.device('cuda'...)"
assert "NUM_CLASSES" in globals(), "NUM_CLASSES not found. Make sure NUM_CLASSES and LABELS are defined."

model = _build_model_from_notebook_or_ckpt(ckpt)
model = model.to(device)

print("Model type:", type(model).__name__)


AssertionError: device not found. Make sure you defined: device = torch.device('cuda'...)

In [5]:
# ============================================
# CELL 4: Load weights + apply EMA if present
# ============================================
def _get_state_dict(ckpt: dict):
    if "model_state" in ckpt and isinstance(ckpt["model_state"], dict):
        return ckpt["model_state"]
    if "state_dict" in ckpt and isinstance(ckpt["state_dict"], dict):
        return ckpt["state_dict"]
    # sometimes saved as "model"->state_dict only, but we already tried model object above
    raise KeyError("No 'model_state' or 'state_dict' found in checkpoint.")

sd = _get_state_dict(ckpt)

missing, unexpected = model.load_state_dict(sd, strict=False)
print("Loaded weights.")
print("  missing keys   :", len(missing))
print("  unexpected keys:", len(unexpected))

# Apply EMA shadow weights if present
use_ema = bool(ckpt.get("use_ema", False))
ema_shadow = ckpt.get("ema_shadow", None)

if use_ema and isinstance(ema_shadow, dict) and len(ema_shadow) > 0:
    with torch.no_grad():
        curr = model.state_dict()
        applied = 0
        for k, v in ema_shadow.items():
            if k in curr:
                curr[k].copy_(v.to(curr[k].device).to(curr[k].dtype))
                applied += 1
        model.load_state_dict(curr, strict=False)
    print(f"Applied EMA shadow weights: {applied} tensors")
else:
    print("EMA weights not found / not used.")

model.eval()


NameError: name 'model' is not defined

In [24]:
# ============================================================
# CELL 5: Evaluate (uses your existing eval_sequence function)
# ============================================================
assert "eval_sequence" in globals(), (
    "eval_sequence() not found in globals.\n"
    "Run the cell where you defined eval_sequence before this evaluation cell."
)

def _print_metrics(tag, m):
    print(f"\n===== {tag} =====")
    for k in ["loss","acc","macro_f1","kappa","AUROC","AUPRC","meanConf","ECE"]:
        if k in m:
            print(f"{k:9s}: {m[k]:.4f}")
    if "bad_batches" in m:
        print("bad_batches:", m["bad_batches"])
    if "f1_per_class" in m:
        print("F1/class :", m["f1_per_class"])
    if "cm" in m:
        labs = [LABELS[i] for i in range(NUM_CLASSES)] if "LABELS" in globals() else list(range(NUM_CLASSES))
        print("Confusion Matrix (rows=true, cols=pred) labels:", labs)
        print(m["cm"])

# Required loaders must exist
for nm in ["val_seq_loader", "test_seq_loader", "ext_seq_loader"]:
    assert nm in globals(), f"{nm} not found. Please create it before running evaluation."

val_m  = eval_sequence(model, val_seq_loader,  desc="VAL")
test_m = eval_sequence(model, test_seq_loader, desc="SHHS1 TEST")
ext_m  = eval_sequence(model, ext_seq_loader,  desc="SHHS2 EXT")

_print_metrics("VAL", val_m)
_print_metrics("SHHS1 TEST", test_m)
_print_metrics("SHHS2 EXT", ext_m)

# Optional MESA
if "mesa_seq_loader" in globals():
    mesa_m = eval_sequence(model, mesa_seq_loader, desc="MESA EXT")
    _print_metrics("MESA EXT", mesa_m)
else:
    print("\n[MESA] mesa_seq_loader not found. (Skip)")



===== VAL =====
loss     : 0.8704
acc      : 0.8593
macro_f1 : 0.8023
kappa    : 0.8048
AUROC    : 0.9705
AUPRC    : 0.8486
meanConf : 0.6920
ECE      : 0.1673
bad_batches: 0
F1/class : {'W': 0.9133298332330255, 'N1': 0.5192242833052276, 'N2': 0.8697101058497362, 'N3': 0.8263279694311394, 'REM': 0.8830118850982974}
Confusion Matrix (rows=true, cols=pred) labels: ['W', 'N1', 'N2', 'N3', 'REM']
[[106987   5595   4747    501   1808]
 [  2458  12316   3538      8   1715]
 [  4002   7452 188691  14192   8814]
 [   238     14  10567  61416    147]
 [   956   2028   3223    149  71101]]

===== SHHS1 TEST =====
loss     : 0.8333
acc      : 0.8686
macro_f1 : 0.8119
kappa    : 0.8175
AUROC    : 0.9738
AUPRC    : 0.8619
meanConf : 0.6915
ECE      : 0.1771
bad_batches: 0
F1/class : {'W': 0.9209574694378058, 'N1': 0.5344476681245398, 'N2': 0.8784149801573615, 'N3': 0.8341778076226524, 'REM': 0.8915967122465064}
Confusion Matrix (rows=true, cols=pred) labels: ['W', 'N1', 'N2', 'N3', 'REM']
[[110403


===== MESA EXT =====
loss     : 1.4322
acc      : 0.7700
macro_f1 : 0.6181
kappa    : 0.6656
AUROC    : 0.9228
AUPRC    : 0.7178
meanConf : 0.6281
ECE      : 0.1419
bad_batches: 0
F1/class : {'W': 0.8713183480964034, 'N1': 0.38780251316123404, 'N2': 0.7856198036341492, 'N3': 0.24597317049716316, 'REM': 0.7999503681321465}
Confusion Matrix (rows=true, cols=pred) labels: ['W', 'N1', 'N2', 'N3', 'REM']
[[685027  22805  80228    885  41018]
 [ 24112  59594  84645     18  18821]
 [ 24274  33508 708261   4150  27705]
 [  1500     55 117830  20463    929]
 [  7516   4190  14201     91 228871]]
